In [56]:
Zx.<x> = ZZ[]
Qx.<y> = QQ[]
n = 4
sigmamin = 1.277833697
sigmamax = 1.8205
q = 12*1024+1
phi = x^n+1
from os import urandom
from Crypto.Hash import SHAKE256
LN2 = 0.69314718056
sigma_star = 1.17 / sqrt(q / (2*n)) 
SALT_LEN = 40


In [57]:
def InnerProduct(a,b,n):
    s = [a[i]*b[i].conjugate() for i in range(n)]
    return sum(s)

# formula 3.9 from page 24, author: Evgen Postulga
def EuclideanNorm(a,n):
    # return sqrt(InnerProduct(a,a,n))
    b=InnerProduct(a,a,n)
    return sqrt(float(b))

# formula 3.25 from page 30, author: Maxim Pushkar
def FieldNorm(f, n):
    f0, f1 = Split(f,n)
    iks = f.parent()([0, 1])
    return (f0^2 - iks * f1^2) % (iks^(n//2)+1)

# NNT representation of polynomial f from Zq[x]/x^n+1, page 28, author Karina Ilchenko
def NTT(f, n, q):
        
    roots = (x^n + 1).roots(Integers(q))
    ans = [f.subs(x = i[0]) % q for i in roots]
    return ans

In [58]:
def CyclicRotate(input, n):
    return input[-n:] + input[0:-n]

In [59]:
def HermitianAdjointPoly(p, n):
    f=[p[0]]
    for i in range(1,n):    
        f.append(-p[n-i])
    return p.parent()(f)

In [60]:
def LatticeToPoly4(M, n): 
    p00= M[0][:n]
    p01= M[0][n:]
    p10= M[n][:n]
    p11= M[n][n:]
    return p00, p01, p10, p11

In [61]:
def PolyToLattice4(p00, p01, p10, p11, n):
    M=[]
    p=[p00.coefficients(sparse=False),p01.coefficients(sparse=False),p10.coefficients(sparse=False),p11.coefficients(sparse=False)]
    for i in range(4):
        while len(p[i])!=n:
            p[i].append(0)
    for i in range(n):
        m1 = CyclicRotate(p[0], i)
        m2 = CyclicRotate(p[1], i)
        M.append(m1+m2)
    for i in range(n):
        m1 = CyclicRotate(p[2], i)
        m2 = CyclicRotate(p[3], i)
        M.append(m1+m2)
    return M

In [62]:
def make_ok(matrix):
    l1 = matrix[:4]
    for row in matrix[4:]:
        newr = []
        for el in row:
            if '/' in str(el):
                n,d = str(el).split('/')
                newr.append(round(float(float(n)/float(d)),2))
            else:
                newr.append(el)
        l1.append(newr)
    return l1

In [63]:
def Merge(a,b):
    iks = a.parent()([0, 1])
    a = a.subs(iks=iks^2)
    b = iks*b.subs(iks=iks^2)
    return a + b

In [64]:
class Node:
    def __init__(self, value):
        self.value = value
        self.leftchild = 0
        self.rightchild = 0
        self.tree = [self.value,self.leftchild,self.rightchild]
        
    def update_tree(self):
        self.tree = [self.value,self.leftchild,self.rightchild]
        
    def __str__(self):
        return '[' + str(self.value) + ',' + str(self.leftchild) + ','  + str(self.rightchild) + ']' 
    
    def __repr__(self):
        return '[' + str(self.value) + ',' + str(self.leftchild) + ','  + str(self.rightchild) + ']'
    
    def print_tree(self, pref=""):
        leaf = "|—————> "
        top = "|_______"
        son1 = "|       "
        son2 = "        "
        width = len(top)

        a = ""
        if (self.value * y^0).degree() and self.leftchild:
            if (pref == ""):
                a += pref + str(self.value) + "\n"
            else:
                a += pref[:-width] + top + str(self.value) + "\n"
            try:
                a += self.leftchild.print_tree(pref + son1)
                a += self.rightchild.print_tree(pref + son2)
            except:
                pass
            return a
        else:
            return (pref[:-width] + leaf + str(self.value) + "\n")
        
def normalize_tree(tree, sigma):
    if (tree.value * y^0).degree() and tree.leftchild and tree.rightchild:
        normalize_tree(tree.leftchild, sigma)
        normalize_tree(tree.rightchild, sigma)
    else:
        
        tree.value = sigma / sqrt(tree.value.real())
        tree.update_tree()

In [65]:
def Omega(n):
    phi4 = cyclotomic_polynomial(4)
    phi_n = phi4.complex_roots()
    phi_n.reverse()
    k = 2
    while k != n:
        phi_n = sum([[sqrt(elt), - sqrt(elt)] for elt in phi_n], [])
        k = 2*k
    return phi_n

# formula 3.18, page 27, author: Evgen Postulga
def FFT(f, n):
    return [f.subs(k) for k in Omega(n)]


def SplitFFT(f):
    n = len(f)
    w = Omega(n)
    f0 = [0] * (n // 2)
    f1 = [0] * (n // 2)
    for i in range(n // 2):
        f0[i] = 0.5 * (f[2 * i] + f[2 * i + 1])
        f1[i] = 0.5 * (f[2 * i] - f[2 * i + 1]) / w[2 * i]
    return f0, f1


def MergeFFT(f0, f1):
    n = 2 * len(f0)
    w = Omega(n)
    f = [0] * n
    for i in range(n // 2):
        f[2 * i + 0] = f0[i] + w[2 * i] * f1[i]
        f[2 * i + 1] = f0[i] - w[2 * i] * f1[i]
    return f
def invFFT(f_fft):
    n = len(f_fft)
    if (n > 2):
        f0_fft, f1_fft = SplitFFT(f_fft)
        f0 = invFFT(f0_fft)
        f1 = invFFT(f1_fft)

        f = n*[0]
        for i in range(n//2):
            f[2*i+0] = f0[i]
            f[2*i+1] = f1[i] 

    elif (n == 2):
        f = [0, 0]
        f[0] = f_fft[0].real()
        f[1] = f_fft[0].imag()
    return f

In [66]:
def InnerProduct(a,b,n):
    s = [a[i]*b[i].conjugate() for i in range(n)]
    return sum(s)


def EuclideanNorm(a,n):
    
    b=InnerProduct(a,a,n)
    return sqrt(float(b))


def FieldNorm(f, n):
    f0, f1 = Split(f,n)
    iks = f.parent()([0, 1])
    return (f0^2 - iks * f1^2) % (iks^(n//2)+1)


def NTT(f, n, q):
        
    roots = (x^n + 1).roots(Integers(q))
    ans = [f.subs(x = i[0]) % q for i in roots]
    return ans

In [67]:
def add_fft(f, g):
    #print("f+g",f,g)
    assert len(f) == len(g)
    deg = len(f)
    return [f[i] + g[i] for i in range(deg)]

def neg_fft(f):
    deg = len(f)
    return [- f[i] for i in range(deg)]

def sub_fft(f, g):
    return add_fft(f, neg_fft(g))

def mul_fft(f_fft, g_fft):
    deg = len(f_fft)
    return [f_fft[i] * g_fft[i] for i in range(deg)]

def div_fft(f_fft, g_fft):
    assert len(f_fft) == len(g_fft)
    deg = len(f_fft)
    return [f_fft[i] / g_fft[i] for i in range(deg)]

In [68]:
def UniformBits(k):
    return int.from_bytes(bytes(list(floor(uniform(0, 256)) for i in range(k / 8))), 'big')

def BaseSampler():
    u = UniformBits(72) 
    z_0 = 0
    RCDT = [3024686241123004913666, 1564742784480091954050, 636254429462080897535, 199560484645026482916, 47667343854657281903, 8595902006365044063, 1163297957344668388, 117656387352093658, 8867391802663976, 496969357462633, 20680885154299, 638331848991, 14602316184, 247426747, 3104126, 28824, 198, 1, 0]
    for i in range(0, 18):
        z_0 = z_0 + int(u<RCDT[i]) 
    return z_0

def ApproxExp(x, ccs):
    C = [0x00000004741183A3,0x00000036548CFC06,0x0000024FDCBF140A,0x0000171D939DE045,0x0000D00CF58F6F84,0x000680681CF796E3,0x002D82D8305B0FEA,0x011111110E066FD0,0x0555555555070F00,0x155555555581FF00,0x400000000002B400,0x7FFFFFFFFFFF4800,0x8000000000000000]
    y = C[0]
    z = floor(2^63*x)
    for i in range(1, 13):
        y = C[i] - (z*y) >> 63
    z = floor(2^63*ccs)
    y = (z*y) >> 63
    return y

def BerExp(x, ccs):
    s = floor(x/LN2) 
    r = x - s*LN2 
    s = min(s, 63)
    z = (2*ApproxExp(r, ccs) - 1) >> s
    for i in range(56, -8, -8):
        p = UniformBits(8) 
        w = p - ((z >> i) & 0xFF)
        if int(w) == 0:
            break
    return int(w < 0)

def SamplerZ(mu, sigma, sigmamin, sigmamax):
    r = mu - int(floor(mu)) 
    ccs = sigmamin/sigma
    while True:
        z_0 = BaseSampler()
        b = UniformBits(8)&0x1 
        z = b + (2*b-1)*z_0
        x = (z-r)^2/2/sigma^2 - z_0^2/2/sigmamax^2
        if BerExp(x, ccs) == 1:
            return z + int(floor(mu))

In [69]:
def LDL(g00, g01, g10, g11, n):
    D00 = g00
    g100 = FFT(g10,n)
    g000 = FFT(g00,n)
    L10 = div_fft(g100,g000)
    L10_star = HermitianAdjointPoly(Qx(invFFT(L10)),n)
    D11 = sub_fft(FFT(g11,n), mul_fft(mul_fft(L10,FFT(L10_star,n)),g000))
    L = PolyToLattice4(1*y^0,0*y^0,Qx(invFFT(L10)),1*y^0,n)
    D = PolyToLattice4(D00,0*x^0,0*x^0,Qx(invFFT(D11)),n)
    return L,D

In [70]:
def ffLDL(g00, g01, g10, g11, n):
    L,D = LDL(Qx(g00), Qx(g01), Qx(g10), Qx(g11), n)
    L10 = Qx(LatticeToPoly4(L,n)[2])
    tree = Node(L10)
    D00 = Qx(LatticeToPoly4(D,n)[0])
    D11 = Qx(LatticeToPoly4(D,n)[3])
    if n == 2:
        tree.leftchild = Node(FFT(D00,n)[0])
        tree.rightchild = Node(FFT(D11,n)[0])
        tree.update_tree()
       
        return tree
    else:
        d00,d01 = SplitFFT(FFT(D00,n))
        d10,d11 = SplitFFT(FFT(D11,n))
        d01_star = HermitianAdjointPoly(Qx(invFFT(d01)),n//2)
        d11_star = HermitianAdjointPoly(Qx(invFFT(d00)),n//2)
        tree.leftchild = ffLDL(invFFT(d00),invFFT(d01),d01_star,invFFT(d00),n//2)
        tree.rightchild = ffLDL(invFFT(d01),invFFT(d11),d11_star,invFFT(d10),n//2)
    return tree   

In [71]:
def ffSampling(t0_, t1_, T, sigmamin, sigmamax, q, n):
    
    if n == 1:
        
        sigma = T
        
        
        z0_ = SamplerZ(t0_[0].real(),float(sigma.value.real()),sigmamin, sigmamax)
        z1_ = SamplerZ(t1_[0].real(),float(sigma.value.real()),sigmamin, sigmamax)
        
        return [z0_],[z1_]
  
    l = T.value
    T0 = T.leftchild
    T1 = T.rightchild
    t1 = SplitFFT(t1_)
    z1 = ffSampling(*t1, T1, sigmamin, sigmamax, q, n/2)
    z1_ = MergeFFT(*z1)
    t0__ = add_fft(t0_, mul_fft(sub_fft(t1_,z1_),l))
    t0 = SplitFFT(t0__)
    z0 = ffSampling(*t0, T0, sigmamin, sigmamax, q, n/2)
    z0_ = MergeFFT(*z0)
    return z0_,z1_

In [72]:
def Balance(f,q,n):
    g = list(((f[i] + q//2) % q) - q//2 for i in range(n))
    return f.parent()(g)


def Split(f,n):
    f0 = list(f[2*i+0] for i in range(n//2))
    f1 = list(f[2*i+1] for i in range(n//2))
    return f.parent()(f0), f.parent()(f1)  

In [73]:
def Reduce(f, g, F, G, n):
    
    TT = Zx.change_ring(QQ).quotient(x^n+1) 
    
    f_star = HermitianAdjointPoly(f, n)
    g_star = HermitianAdjointPoly(g, n)
    while True:
        num = F*f_star + G*g_star
        num = TT(num)
        den = f*f_star + g*g_star
        den = 1 / TT(den)
        res = num * den
        k = Zx([int(round(elt)) for elt in res])
        F = F - k*f 
        G = G - k*g
        if all(elt == 0 for elt in k):
            break
    return f, g, F, G

def NTT(f, n, q):
  
    roots = (x^n + 1).roots(Integers(q))
    ans = [f.subs(x = i[0]) % q for i in roots]
    return ans

In [74]:
def NTRUSolve(f, g, n, q):
    if n == 1:
       
        gcd_, u, v = xgcd(f[0], g[0])
        
        if gcd_ != 1:
            return None, None, False
        F, G = -v*q, u*q
       
        return F, G, True
    else:
        
    
        f_ = FieldNorm(f, n) 
        g_ = FieldNorm(g, n) 
        
        F_, G_, flag = NTRUSolve(f_, g_, n//2, q)
        if flag:
            F = F_.subs(x=x^2) * g.subs(x=-x) 
            G = G_.subs(x=x^2) * f.subs(x=-x)
            
            f, g, F, G = Reduce(f, g, F, G, n)
            return F % (x^n +1), G % (x^n +1), flag
        else:
            return F_, G_, flag

In [75]:
def HashToPoint(salt, message, q, n):

    k = int((2**16) // q)
    
    shake = SHAKE256.new()
    shake.update(salt)
    shake.update(message)
    hashed = [0 for i in range(n)]
    
    i = 0
    j = 0
    while i < n:
        twobytes = shake.read(2)
        elt = (twobytes[0] << 8) + twobytes[1] 
        if elt < k * q:
            hashed[i] = elt % q
            i += 1
        j += 1
    return Zx(hashed) % phi

In [76]:
def NTRUGen(q, n):
    
    def gen_poly(n, q):
        
        def D(mu=0):
            z = 0
            for i in range(1, 4096/n + 1):
                sigma_star = 1.17 * sqrt(q / 8192)
                sigmamin, sigmamax = 1.277833697, 1.8205
                zi = SamplerZ(mu, sigma_star, sigmamin, sigmamax)
                z += zi
            return z

        f = [0] * n
        g = [0] * n
        for i in range(n):
            f[i] = D()
            g[i] = D()
        f = Zx(f) % phi
        g = Zx(g) % phi
        return f, g
    
    def gs_norm(f, g, q, n):
        TT = Zx.change_ring(QQ).quotient(x^n+1) 
            
        f_star = HermitianAdjointPoly(f, n)
        g_star = HermitianAdjointPoly(g, n)
        first = EuclideanNorm([*g.coefficients(sparse=False), *(-f).coefficients(sparse=False)], n)
        s1 = (q * TT(f_star)) / TT((f*f_star + g*g_star))
        s2 = (q * TT(g_star)) / TT((f*f_star + g*g_star))
        second = EuclideanNorm(list(s1) + list(s2), n)
        gamma = max(first, second)
        return gamma
    while True:
        while True:
            while True:

                f, g = gen_poly(n, q)
                

                if gs_norm(f, g, q, n) > (1.17 ** 2) * q:
                    
                    continue
                break

            if  0 in NTT(f, n, q):
               
                continue
            break
                
        F, G, flag = NTRUSolve(f, g, n, q)
        
        if not flag:
            
            continue
        else:
            F, G = F % (x^n +1), G % (x^n +1)
            F = Zx([int(coef) for coef in F.coefficients(sparse=False)])
            G = Zx([int(coef) for coef in G.coefficients(sparse=False)])
            
            break
            
    return f, g, F, G


In [77]:
def NaiveKeyGen(sigma, q, n):
    f, g, F, G = NTRUGen(q, n)
    B = [[g, -f], [G, -F]]
    sk = B  
    
   
    
    TT = Zx.change_ring(Integers(q)).quotient(x^n+1)
    f_q = Zx(lift(1 / TT(f))) 
    
    h = g * f_q % phi % q
    h_star = HermitianAdjointPoly(h, n)
    A = [Zx(1), h_star]
    pk = A
    
    
    A_star = [[Zx(1)], [h]]
    r00 = (B[0][0] * A_star[0][0] + B[0][1] * A_star[1][0]) % phi % q 
    r10 = (B[1][0] * A_star[0][0] + B[1][1] * A_star[1][0]) % phi % q 
    result = [[r00], [r10]]
    
    
    return sk, h

In [78]:
def NaiveSign(message, B, q, n):
    
    noise = urandom(SALT_LEN)
    
    
    c = HashToPoint(noise, message, q, n)
   
    B_ = [[B[1][1], -B[0][1]], [-B[1][0], B[0][0]]]
    
    B_inv = [[B[1][1]/q, -B[0][1]/q], [-B[1][0]/q, B[0][0]/q]]
    B_ = [[B[1][1], -B[0][1]], [-B[1][0], B[0][0]]]
    B_inv = [[B[1][1]/q, -B[0][1]/q], [-B[1][0]/q, B[0][0]/q]]

    

    
    t = [c.coefficients(sparse=False), [[0]*n]]
    
    t00 = c * B_inv[0][0]  % phi
    t01 = c * B_inv[1][0]  % phi
    t = t00.coefficients(sparse=False) + t01.coefficients(sparse=False)
    
    z = [int(round(float(el))) for el in t]
    
     
    
    z1 = Zx(list(z)[:n])
    z2 = Zx(list(z)[n:])
    zb1 = (z1 * B[0][0] + z2 * B[1][0]) % phi
    zb2 = (z1 * B[0][1] + z2 * B[1][1]) % phi
    zb1 = zb1.coefficients(sparse=False)
    zb2 = zb2.coefficients(sparse=False)
    zb = vector(zb1 + zb2)
    
    
    c0 = vector(c.coefficients(sparse=False) + [0]*n)
    
    s = c0 - zb
    beta = EuclideanNorm(Balance(s1, q, n).coefficients(sparse=False) + Balance(s2, q, n).coefficients(sparse=False), n)
    

    sa = (s1 * Zx(1) + s2 * h) % (x^n + 1) % q

    return noise, s2, beta


In [79]:
def NaiveVerify(message, noise, s2, h, q, n, beta):
    c = HashToPoint(noise, message, q, n)
    s1 = (c - s2 * h) % phi
    if EuclideanNorm(Balance(s1, q, n).coefficients(sparse=False) + Balance(s2, q, n).coefficients(sparse=False), n) == beta:
        return True
    else:
        return False